In [1]:
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

Загружаем данные из файлов

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

Посмотрим на наши данные

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Здесь можно видно, что в данных есть колонки, которые нам не нужны и мы их в дальнейшем просто удалим, а некоторые нам понадобятся для другого. В частности колонка `Survived` нам нужна для обучения, а `PassengerId` для подготовки файла, который буду отправлять на Kaggle https://www.kaggle.com/c/titanic/data 

### Предобработка данных.

Фиксируем целевую переменную, после чего удалим ее из `df_train`

In [4]:
labels = df_train["Survived"]
# Удаляем целевую переменную
df_train.drop("Survived", axis=1, inplace=True)

Сохраняем `PassengerId`, эти данные нужны для того, чтобы создать файл для посылки на Kaggle.

In [5]:
data_passengerids = df_test["PassengerId"]

 Удалим признаки `PassengerId`, `Name`, `Fare`, `Ticket` и `Cabin` из данных.

In [6]:
columns_to_drop = ["Ticket", "PassengerId", "Name", "Cabin", "Fare"]

df_train.drop(columns_to_drop, axis=1, inplace=True)
df_test.drop(columns_to_drop, axis=1, inplace=True)

В наших данных есть слова, а не цифры, поэтому заменим эти данные.

In [7]:
def replace_obj(data):
    data["Sex"] = (data["Sex"] == "female").astype(int)
    data["Embarked"] = data["Embarked"].map({"S":1, "C":2, "Q":3})

replace_obj(df_train)
replace_obj(df_test)

После обработки нужно посмотреть где есть пропуски.

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass      891 non-null int64
Sex         891 non-null int64
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Embarked    889 non-null float64
dtypes: float64(2), int64(4)
memory usage: 41.8 KB


In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
Pclass      418 non-null int64
Sex         418 non-null int64
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Embarked    418 non-null int64
dtypes: float64(1), int64(5)
memory usage: 19.7 KB


В обучающей выборке есть пропуски в признаках `Age` и `Embarked`,в тестовой - в `Age`.
Заполним пропуски в признаке `Age` обучающей и тестовой выборок медианным значением. Пропуски в признаке `Embarked` обучающей выборки самыми частыми значениями этого признака.

In [10]:
# Заменяем медианной из данных df_train, т.к. здесь будет более точное значение.
df_train['Age'].fillna(df_train['Age'].median(), inplace=True)
df_test['Age'].fillna(df_train['Age'].median(), inplace=True)
df_train['Embarked'].fillna(df_train['Embarked'].value_counts().idxmax(), inplace=True)

### Обучиние классификатора kNN и оценка качества с помощью cross_val_score

In [11]:
knn = KNeighborsClassifier()
# Оценка качества
cross_val_scores = cross_val_score(knn, df_train, labels, scoring="accuracy", cv=5)
print(cross_val_scores.mean(), cross_val_scores.std())

0.7722276895382524 0.022779090578995347


Достаточно хороший показатель. Я решил не создавать новые признаки, которые можно создать перемножив один признак на другой, а просто посмотреть, что будет, если убрать те признаки, которые имеют малый вес. После проб решил оставить только 3 признака.

In [12]:
df_train.drop(["Age", "Parch", "SibSp"], axis=1, inplace=True)
df_test.drop(["Age", "Parch", "SibSp"], axis=1, inplace=True)

Улучшаем показатель с помощью GridSearchCV

In [13]:
# Параметры knn для GridSearchCV
knn_params = {'n_neighbors': list(range(1, 30)), 
               'weights': ['uniform','distance'],
               'leaf_size':list(range(1, 30))}

knn_grid = GridSearchCV(knn, knn_params, cv=5, scoring='accuracy')
knn_grid.fit(df_train, labels)

knn_grid.best_params_

/home/mif/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'leaf_size': 6, 'n_neighbors': 28, 'weights': 'distance'}

In [14]:
knn = KNeighborsClassifier(n_neighbors=28, weights='distance', leaf_size=6)
cross_val_scores = cross_val_score(knn, df_train, labels, scoring="accuracy", cv=5)
print(cross_val_scores.mean(), cross_val_scores.std())

0.8114470348292265 0.015302930516544523


### Подготовка файла для отправки на Kaggle.

In [15]:
knn = KNeighborsClassifier(n_neighbors=28, weights='distance', leaf_size=6)
knn.fit(df_train, labels)
predictions = knn.predict(df_test)

def save_submission(predictions, test_idx, fout="knn_titanic.csv"):
    submission = pd.DataFrame({"PassengerId":test_idx, "Survived":predictions})
    submission.to_csv(fout, index=False)

save_submission(predictions, test_idx=data_passengerids)

### Результа на Kaggle

![Screenshot](screenshot_titanic_kaggle_knn.png "Скриншот с сайта Kaggle")